In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [2]:
GROQ_API_KEY= ""

In [3]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [4]:
loader = PyPDFDirectoryLoader(r"C:/Users/HP/Desktop/Projects/InsuranceTracker/Health Insurance/")

In [5]:
docs = loader.load()

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
)

In [7]:
documents = text_splitter.split_documents(docs)

In [8]:
len(documents)

528

In [12]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:········


In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [14]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def health_insurance_RAG(query):
    prompt = hub.pull("rlm/rag-prompt")
    llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="llama3-70b-8192")
    retriever = db.as_retriever(search_kwargs={"k": 10})
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    output = rag_chain.invoke(query)
    return output

In [15]:
health_insurance_RAG("What is the coverage for vaccinations under TATA AIG Health Insurance, and up to what amount are they covered?")

'The coverage for vaccinations under TATA AIG Health Insurance is as follows:\n\n* First-year vaccinations are covered up to `10,000 (`15,000 for a girl child) provided the maternity claim is admitted.\n* Vaccination cover is available for certain vaccines, including Anti-rabies vaccine following an animal bite, Typhoid vaccination, Human Papilloma Virus (HPV) vaccine, and Hepatitis B Vaccine, without any waiting period for sums insured up to `50 Lakhs, and after 2 years of continuous coverage for sums insured above `50 Lakhs.'

In [16]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI
def PremiumTracker(query):
    agent = create_csv_agent(
        ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        r"C:\Users\HP\Desktop\Projects\InsuranceTracker\Health Insurance\Premium.csv",
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )
    output = agent.run(query)
    return output

In [17]:
PremiumTracker("Which Insurance Plan has the lowest 1-Year Premium")

C:\Users\HP\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Premium_1Year'] == df['Premium_1Year'].min()]['Insurance Plan']"}`


0    Aspire Gold+ Combo
Name: Insurance Plan, dtype: objectThe insurance plan with the lowest 1-Year Premium is "Aspire Gold+ Combo".

> Finished chain.


'The insurance plan with the lowest 1-Year Premium is "Aspire Gold+ Combo".'

In [18]:
from langchain.agents import Tool
from langchain.tools import BaseTool
policy_document_tool = Tool(
    name = "content",
    func = health_insurance_RAG,
    description= "use to answer user questions related to policy rules, regulations, and other aspects of health insurance policy documents."
)

In [19]:
premium_trackr_tool = Tool(
    name = "premium",
    func = PremiumTracker,
    description= "use to query Premium related queries for Health Insurance Plans"
)

In [20]:
tools = [policy_document_tool, premium_trackr_tool]

## creating Agent

In [21]:
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferWindowMemory

In [75]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a very powerful assistant. Your job is to answer user-specific questions. you should first check the intent of the question. If the intent is specifically about premium, it should refer to the premium tool. For other queries, it should refer to the content tool.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [76]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.5)

In [77]:
llm_with_tools = llm.bind_tools(tools)

In [78]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [79]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a very powerful assistant. Your job is to answer user-specific questions. you should first check the intent of the question. If the intent is specifically about premium, it should refer to the premium tool. For other queries, it should refer to the content tool.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])

In [80]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)

In [81]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable={
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a very powerful assistant. Your job is to answer user-specific questions. you should first check the intent of the question. If the intent is specifically about premium, it should refer to the premium tool. For other queries, it should refer to the content tool.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), Mess

In [82]:
result = list(agent_executor.stream({"input": "Provide all the details on cancer-related coverage policies for HDFC ERGO Optima Secure in less than 2 sentences. What is 3 Year premium for HDFC ERGO Optima Secure. Summarize the key insights in bullet points."}))



> Entering new AgentExecutor chain...

Invoking: `content` with `cancer-related coverage policies for HDFC ERGO Optima Secure`


HDFC ERGO Optima Secure policy provides cancer-related coverage under its base coverage, which includes hospitalization expenses, day care treatment, and domiciliary hospitalization. Additionally, it offers optional covers such as my: health Critical Illness Add On, which provides a lump sum payment in case of diagnosis of a critical illness, including cancer.
Invoking: `premium` with `3 Year premium for HDFC ERGO Optima Secure`




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Insurance Plan'] == 'Optima Secur']['Premium_3Year'].values[0]"}`


68401The 3-year premium for HDFC ERGO Optima Secure is 68,401.

> Finished chain.
The 3-year premium for HDFC ERGO Optima Secure is 68,401.Here are the key details and insights:

- **Cancer-Related Coverage in HDFC ERGO Optima Secure**:
  - Includes hospitalization expense

In [85]:
# The key we want to check for
first_key = 'output'
# Filter the list to include only dictionaries that have the first_key
filtered_data = [d for d in result if first_key in d]

In [86]:
filtered_data[0]['output']

'Here are the key details and insights:\n\n- **Cancer-Related Coverage in HDFC ERGO Optima Secure**:\n  - Includes hospitalization expenses, day care treatment, and domiciliary hospitalization under its base coverage.\n  - Offers an optional cover, my: health Critical Illness Add On, which provides a lump sum payment upon diagnosis of critical illnesses including cancer.\n\n- **3-Year Premium for HDFC ERGO Optima Secure**:\n  - The premium amount for a three-year term is ₹68,401.'